# 연간고객구매데이터_인사이트
: 아래 가이드에 따라서 고객별 연간 구매데이터를 생성하고 아래사항을 수행하세요(관측치 1000개)

## Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

거래 번호
- id : 1 ~ 1000번 일련번호 - 개시 후 1천 건의 거래가 있었고 걍 거기에 넘버링한 거라고 이해해보자

고객별 속성 데이터
- cusno : 고객번호 1000개 랜덤
- gender : 0,1 정수 난수 생성
- age : 0 ~ 80사이 정수 난수 생성
- region : 1 ~ 10 사이 정수 난수 생성

거래 데이터

- product_20 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- product_21 : 고가제품(10), 중가제품(20), 저가제품(30)로 구분제품코드 10,20,30
- price_20 : 1000 ~ 50000 사이 실수 난수 생성
- price_21 : 1000 ~ 50000 사이 실수 난수 생성
- quantity_20 : 1 ~ 100 사이 정수 난수 생성
- quantity_21 : 1 ~ 100 사이 정수 난수 생성

- amount_20 : price_20 * quantity_20
- amount_21 : price_21 * quantity_21
- sales : 20년 대비 21년구매금액이 증가면 1, 감소면 0

In [57]:
from numpy.random import randint as rri

id_ = np.array(range(1, 1001))

cusno = rri(1000, size = 1000)

gender = rri(2, size = 1000)
age = rri(81, size = 1000)
region = rri(1, 11, size = 1000)

In [58]:
product_20 = (rri(1, 4, size = 1000))*10
product_21 = (rri(1, 4, size=1000))*10
price_20 = np.random.uniform(1000.0, 50000.0, size=1000)
price_21 = np.random.uniform(1000.0, 50000.0, size=1000)
quantity_20 = rri(1, 101, size=1000)
quantity_21 = rri(1, 101, size=1000)

In [59]:
amount_20 = price_20*quantity_20
amount_21 = price_21*quantity_21

sales =np.where((amount_20 < amount_21), 1, 0)

In [60]:
data = [id_, cusno, gender, age, region, 
        product_20, product_21,price_20, price_21, quantity_20, quantity_21,
       amount_20, amount_21, sales]
sr_list = []
for i in data:
    i = pd.Series(i)
    sr_list.append(i)
type(sr_list), type(sr_list[0])

(list, pandas.core.series.Series)

In [61]:
df = pd.concat(sr_list, axis=1)
list_col = ['id_', 'cusno', 'gender', 'age', 'region', 
        'product_20', 'product_21', 'price_20', 'price_21', 'quantity_20', 'quantity_21',
       'amount_20', 'amount_21', 'sales']
dict_col = {}
for i in range(len(list_col)):
    dict_col[i] = list_col[i]
df.rename(columns = dict_col, inplace=True)
df.head()

,id_,cusno,gender,age,region,product_20,product_21,price_20,price_21,quantity_20,quantity_21,amount_20,amount_21,sales
0,1,260,1,51,5,20,20,47535.012293,46056.742900,24,26,1.140840e+06,1.197475e+06,1
1,2,680,1,14,6,10,30,13035.189253,15966.202594,62,10,8.081817e+05,1.596620e+05,0
2,3,961,0,80,9,10,10,26324.126659,20975.558691,38,6,1.000317e+06,1.258534e+05,0
3,4,685,1,17,8,10,20,22813.065746,13610.844233,7,34,1.596915e+05,4.627687e+05,1
4,5,754,1,33,4,10,10,45240.472765,46142.577851,5,34,2.262024e+05,1.568848e+06,1


In [62]:
len(set(cusno))

636

## Q2. df를 수정하여 3가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요

### 일단 말이되는 데이터 만들기

- age는 17 ~ 89로 하고 age_cat을 만들어서 categorize : 10대/20대/.../70대/80대, 00 decades로 하자
- region도 분류에 의미부여
    - 1안: 서울/ 경기/ 대전/ 광주/ 부산/ 춘천 ...충청도 전라도 경상도 강원도...
    - 2안: 도시/ 시골 같은...
    - 3안: 가상의 도시 cityA, B, C 생성 (
        - 인구수 각 6만/3만/1만으로 상정, 각각 대도시, 부촌, 농촌 같은 느낌으로 생각
- price는 1원단위 사용X, 정수형/ 10원~50000원
- price에 따라 product를 결정 
    - 구간 30 : ~10000/20 : ~30000/10:  ~50000
- price가 변경되었으니 amount도 변경

### 전처리

- grade_20, grade_21 항목 추가 : 'A', 'B', 'C'/  
    - amount_20, amount_21로 각각 100만 이상/ 30만 이상/ 30만 미만
- amount_inc: amount_21-amount_20/
    - 증가시 +, 감소시 -, sales 항목과 다른 점은 증감한 매출을 알 수 있다는 점

# 해야할 것이 이게 맞는지 확인 받기

## 1천건의 '고객 데이터' 생성

    - cusno : 고객번호 1000개
    - gender : 0,1 정수 난수 생성
    - age : 0 ~ 80사이 정수 난수 생성
    - region : 1 ~ 10 사이 정수 난수 생성
    - 

## 1만건의 '거래 데이터' 생성

    - 거래 no. :  0~9999
    - date : '2020-01-01' ~ '2021-12-31' 기간으로
    - day : 'Mon', ... , 'Sun'
    - time : '00:00' ~ '23:59'
    - 고객 id : 0 ~ 999
    - product : 10:고가, 20: 중가, 30: 저가
    - price : 1000~50000, 1원단위 사용하지 않는 정수
    - quantity : 1~100
    - amount: price * quantity

- 이 랜덤하게 만든 데이터를 요약해서 살펴보고, 인사이트 뭐넣을지 생각하지
- 랜덤 데이터를 왜 들여다 모르겠지만 일단 그렇게 하라고.?.?

    
## 위 두개를 합쳐서(???) '통합 데이터 ' 생성
- 이 두개를 단순히 customer no. 로 join하는거?? 같은데???
    
    - 거래 no. :  0~9999
    - date : '2020-01-01' ~ '2021-12-31' 기간으로
    - day : 'Mon', ... , 'Sun'
    - time : '00:00' ~ '23:59'
    - 고객 id : 0 ~ 999
    - product : 10:고가, 20: 중가, 30: 저가
    - price : 1000~50000, 1원단위 사용하지 않는 정수
    - quantity : 1~100
    - amount: price * quantity
    

    - cusno : 고객번호 1000개
    - gender : 0,1 정수 난수 생성
    - age : 0 ~ 80사이 정수 난수 생성
    - region : 1 ~ 10 사이 정수 난수 생성



## 인사이트를 정한다
    - ex) 코로나로 남성보다 외출 쇼핑 많이 하는 여성 매출 대폭 감소
    - 독립변수 - 성별/  종속변수 - 매출증감여부 / + alpha : 이유를 추측 (이 과제에서는 자기가 story를 만든다?)

## 인사이트가 나오도록 데이터 조작
    - 여기에서 어떤 데이터를 조작해서 인사이트를 도출할 수 있도록 만드나?
        - 통합데이터를 조작해서 이걸로 나중에 만들 고객별 거래데이터가 조작된 상태로 나오도록 조작하라는 것? -> 이거 아주 어려울텐데


### 인사이트 3개 정하기

- 각 도시에서의 매출 변화 : B 도시 매출이 감소
    - 50% 정도의 B도시 사람들의 매출을 30% 감소 시키자.
- grade_20 = 'A' 였던 그룹의  매출 감소, 
    - 해당 그룹의 70%의 사람들의 매출이 30% 정도 감소
    - 21년 매출이 아예 없어지는 항목도 유의미한 정도로 추가
    - 해당 그룹 사람들의 60%는 B 도시에 살도록
    - 

### 인사이트에 맞게 조작

## Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.

# 구매감소고객예측

- 1만건의 거래 데이터
    - 거래 no. :  1~10000
    - date : '2022-01-01' ~ '2022-06-30' 기간으로
    - day : 'Mon', ... , 'Sun'
    - time : '00:00' ~ '23:59'
    - 고객 id : 0 ~ 999
    - (product : 10:고가, 20: 중가, 30: 저가)
    - price : 10~50000, 1원단위 사용하지 않는 정수
    - quantity : 1~100
    - amount: price * quantity